<a href="https://colab.research.google.com/github/rayoen0/data-analytics-with-python-classroom-1a8e1c-streamlit-exercise-Streamlit-Exercise/blob/main/Exercise_Medical_Cost_Prediction_Revisited.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise: Medical cost prediction revisited

Insurance companies need to predict the annual medical cost of a insurance policy holder.

* Target: **annual medical cost**
* Predictors:
    * age, gender, bmi, number of children, smoker/non-smoke, region

## Data Preparation

Let's load the [dataset](https://raw.githubusercontent.com/zhouy185/BUS_O712/refs/heads/main/Data/medical_costs.csv).

In [1]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/zhouy185/BUS_O712/refs/heads/main/Data/medical_costs.csv")
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1332,50,male,30.970,3,no,northwest,10600.54830
1333,18,female,31.920,0,no,northeast,2205.98080
1334,18,female,36.850,0,no,southeast,1629.83350
1335,21,female,25.800,0,no,southwest,2007.94500


We will now seperate the data into X and y, and then perform the train-test split.

In [2]:
# target vs. features
y = df['charges']
X = df.drop('charges',axis=1)

# Train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

Next, construct a column transformer that performs one-hot-encoding

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Define the categorical and numerical features

cat_cols = ['sex', 'smoker', 'region']
num_cols = ['age', 'bmi', 'children']

# Define the column transformer

ct = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(sparse_output=False, drop="first"), cat_cols),
    ],
    remainder="passthrough"  # Pass through numerical columns without scaling
)

# Linear Regression

Let's define a Linear Regression model and include it in a pipeline.

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

# Predictive model
lr = LinearRegression()

# Pipeline
pipe = Pipeline(steps=[("ct", ct),("lr", lr),])

# Define the pipeline
pipe.fit(X_train,y_train)


/usr/local/lib/python3.12/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['sex', 'smoker',
                                                   'region'])])),
                ('lr', LinearRegression())])

Fit the pipeline.

In [5]:
pipe.fit(X_train,y_train)


/usr/local/lib/python3.12/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['sex', 'smoker',
                                                   'region'])])),
                ('lr', LinearRegression())])

Predict on the test data and calculate R2 and RMSE

In [9]:
y_pred = pipe.predict(X_test)

from sklearn.metrics import mean_squared_error, r2_score
import math

mse = mean_squared_error(y_test,y_pred)
rmse = math.sqrt(mse)

r2 = r2_score(y_test,y_pred)

print(f"Root mean squared: {rmse}, R-squared: {r2}")


Root mean squared: 5956.342894363581, R-squared: 0.8069287081198016


# Random Forest Regressor

We now use Random Forest to perform the regression.

In [11]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100, max_depth= 10, min_samples_split = 2, min_samples_leaf=1, random_state=42)

Include the model in the pipeline, and fit the pipeline

In [12]:
pipe = Pipeline(
    steps=[
        ("ct", ct),
        ("rf", rf),
    ]
)

pipe.fit(X_train,y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['sex', 'smoker',
                                                   'region'])])),
                ('rf', RandomForestRegressor(max_depth=10, random_state=42))])

Now predict on test data and evaluate the performance.

In [15]:
y_pred = pipe.predict(X_test)

mse_rf= mean_squared_error(y_test,y_pred)
rmse_rf = math.sqrt(mse)

r2_rf = r2_score(y_test,y_pred)

print(rmse_rf,r2_rf)

4609.089658806331 0.8843918017412256


# XGBoost for Regression

Define the XGBoost Model

In [18]:
from xgboost import XGBRegressor

xgb = XGBRegressor(n_estimators=100, max_depth= 10, subsample = 0.8, colsample_bytree = 0.8, learning_rate=0.01, random_state=42)

Include the model in a pipeline.

In [19]:
pipe = Pipeline(
    steps=[
        ("ct", ct),
        ("xgb", xgb),
    ]
)

pipe.fit(X_train,y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['sex', 'smoker',
                                                   'region'])])),
                ('xgb',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=0.8, device=None,
                              early_stopping_rounds=None,
                              enable_categorical...
                              feature_types=None, feature_weights=None,
                              gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.01,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=10, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=100, n_jobs=None,
                              num_parallel_tree=None, ...))])

Performance evaluation

In [20]:
y_pred = pipe.predict(X_test)

mse = mean_squared_error(y_test,y_pred)
rmse_xgb = math.sqrt(mse)

r2_xgb = r2_score(y_test,y_pred)

print(rmse_xgb,r2_xgb)

8196.080579536421 0.6344298229459825


# LightGBM

Define a LGBM mode.

In [22]:
from lightgbm import LGBMRegressor

lgbm = LGBMRegressor(n_estimators=100, max_depth= 10, subsample = 0.8, colsample_bytree = 0.8, learning_rate=0.01, random_state=42)

Including the model in a pipeline.

In [23]:
pipe = Pipeline(
    steps=[
        ("ct", ct),
        ("lgbm", lgbm),
    ]
)

pipe.fit(X_train,y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000440 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 318
[LightGBM] [Info] Number of data points in the train set: 1069, number of used features: 8
[LightGBM] [Info] Start training from score 13030.203373


/usr/local/lib/python3.12/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['sex', 'smoker',
                                                   'region'])])),
                ('lgbm',
                 LGBMRegressor(colsample_bytree=0.8, learning_rate=0.01,
                               max_depth=10, random_state=42, subsample=0.8))])

Performance evaluation

In [24]:
y_pred = pipe.predict(X_test)

mse = mean_squared_error(y_test,y_pred)
rmse_lgb = math.sqrt(mse)

r2_lgb = r2_score(y_test,y_pred)

print(rmse_lgb,r2_lgb)

7850.492869747294 0.6646084110499411


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


## KNN for Regression

We need the `KNeighborsRegressor()` function from `sklearn.neighbors` to create a KNN model for regression. Recall that we need to specify the number of neighbors.

We will define the KNN model and included in a pipe line

In [27]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=20)

But can we use the same column transformer? Why?

In [28]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler


ct_1he_scale = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(), cat_cols),
        ("num", StandardScaler(), num_cols)
    ],
    remainder="passthrough"  # Pass through numerical columns without scaling
  )

Combine the column transformer and the knn model in a pipeline.

In [29]:
pipe = Pipeline(
    steps=[
        ("ct", ct_1he_scale),
        ("knn", knn),
    ]
)

pipe.fit(X_train,y_train)

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat', OneHotEncoder(),
                                                  ['sex', 'smoker', 'region']),
                                                 ('num', StandardScaler(),
                                                  ['age', 'bmi',
                                                   'children'])])),
                ('knn', KNeighborsRegressor(n_neighbors=20))])

Let's predict on the test data and again compute the RMSE and $R^2$ score.

In [30]:
y_pred = pipe.predict(X_test)

mse = mean_squared_error(y_test,y_pred)
rmse_knn = math.sqrt(mse)

r2_knn = r2_score(y_test,y_pred)

print(rmse_knn,r2_knn)

7655.426448581474 0.6810687341641531


Let's perform a grid search with a 5-fold cross-validation for the KNN regression model.


In [32]:
from sklearn.model_selection import GridSearchCV, KFold

param_grid ={
    'knn__n_neighbors': [5,10,15,20,25,30],

}

kf = KFold()

grid_search = GridSearchCV(
    estimator = pipe,
    param_grid = param_grid,
    scoring = 'r2',
    n_jobs = -1,
    cv = kf
)

grid_search.fit(X_train,y_train)


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('ct',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('cat',
                                                                         OneHotEncoder(),
                                                                         ['sex',
                                                                          'smoker',
                                                                          'region']),
                                                                        ('num',
                                                                         StandardScaler(),
                                                                         ['age',
                                                                          'bmi',
                                                                          'children'])])),
                                       ('knn',
                                        KNeighborsRegressor(n_neighbors=20))]),
             n_jobs=-1,
             param_grid={'knn__n_neighbors': [5, 10, 15, 20, 25, 30]},
             scoring='r2')

Let's retrieve the best KNN regression model by cross-validation, and predict on the test data set.

In [33]:
knn_best = grid_search.best_estimator_

y_pred = knn_best.predict(X_test)

mse = mean_squared_error(y_test,y_pred)
rmse = math.sqrt(mse)
r2 = r2_score(y_test,y_pred)
print(rmse,r2)

6335.152462376414 0.7815900284380415


In [ ]:
grid_search.best_params_

## Among all models we used, which one has the best performance?

Provide your answer below.

In [34]:
"LightGBM!"

'LightGBM!'